#### Teamname: 
#### Score: 0.87571 Kagglename: Ramón Jäckle

#### Namen der Teammitglieder: Marius Urban, Ramón Jäckle, Maen Hanna

Standardcode aus Beispiel um Daten zu erhalten

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wget

In [ ]:
import os
import pandas as pd
import random
import zipfile

In [ ]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

In [ ]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

In [ ]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

## Eingangsformat

Die Modelle erwarten alle ein einheitliches Eingangsformat der Daten. In dem Fall der Bildausschnitte muss das Pixelformat vereinheitlicht werden sowie das Farbformat. Das Eingangs Format kann in verschiedenen Datentypen gespeichert werden. Im ersten Versuch wurde hierfür ein **Numpy-Array** mit RGB-Konvertierung verwendet. Durch die großen Mengen an Daten ist hierbei der Grafikspeicher recht schnell an seine Grenzen gelangt. **Anmerkung vieleicht nicht ganz sachlich genug an die Grenzen gelangt eher durch speicher reichte bei früheren Versuchen nicht aus.** Abhilfe sollen hierbei die im nächsten Abschnitt erläuterten **Datasets** geben.

Die optimale Bildgröße wurde empierisch bestimmt im ersten Versuch wurde mit dem standard Format 224x224 begonnen. Anschließend wurde mit größeren Formaten getestet. Das beste Ergebniss wurde mit dem Format 256x256 erzieht. Erhöt man das Format weiter erziehlt man nur geringe Verbesserungen und erhällt eine längere Laufzeit. **Allgemein wirkt der Teil mit der Bildgrößen hier etwas verloren eher zum nächsten Markdown block verschieben**

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'
val_directory = 'products_leaflets_256/test'

# Bildgröße und Anzahl der Klassen
# 
image_size = (256, 256)
num_classes = len(os.listdir(train_directory))

## Dynamische Eingangsdaten
Wie bereits beschrieben wird durch ein standard Numpy-Array zuviel Speicher verwendet. Aus diesem Grund wird hier ein **Dataset** von Keras verwendet. Diese haben nicht nur den Vorteil die Bilder dynamisch in den Speicher zu laden **Hier vielleicht kurz erklären was dynamisches laden bedeutet.**, sie vereinfachen auch das Laden aus den Verzeichnissen. Während bei dem Numpy-Array durch die Ordenerstruktur iteriert werden muss um die Bilder korrekt einzulesen und korrekt zu labeln, wird ein **Dataset** durch einen eizigen Funktionsaufruf erstellt. Über diesen Funktionsaufruf wird außerdem die Größe des zu vereinheitlichen Eingangsformates eingestellt. Das Resizing der Bilder in das einheitliche Eingangsformat wird dann direkt beim Laden in das **Datasets** durchgeführt.

Weitere Einsellmöglichkeiten wie bilineare Interpolation beim Resizing, durchmischen der Daten um Overfitting zu vermeiden und die **Batch-Size** können ebenfalls eingestellt werden. Der Wert der **Batch-Size** konnte keine Verbesserung erziehlten daher wurde der Standardwert von 32 übernommen.

In [ ]:
# Durchlaufe das Trainingsverzeichnis und lade die Bilder
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

# Durchlaufe das Validierungsverzeichnis und lade die Bilder
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

In [ ]:

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, DenseNet121, DenseNet201, DenseNet169
import tensorflow.keras.optimizers.legacy as legacy_optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

## Netzauswahl

Es wurden verschiedene von Keras bereitgestellte Bildklassifizirungsnetze getestet. Darunter sind Netze wie efficientnet, verschiedene Versionen von ResNet, MobileNet, Densenet und ein selbstgebautes AlexNet. Folgende Netze wurden getestet:

+ EfficientNetV2B0
+ EfficientNetV2B0
+ EfficientNetV2B1
+ ResNet50
+ ResNet101
+ MobileNetV2
+ selbstgebautes Alexnet
+ DenseNet121
+ DenseNet169
+ DenseNet201

Aus dem DenseNet169 resultierte das beste Ergebnis und wurde deshalb gewählt.

Dieses Netz hat vortrainierte Layer und Gewichte diese wurden bei den Standard werten belassen. Durch Testen konnte average Pooling als beste Variante bestimmt werden.

In [ ]:
#base_model = MobileNetV2(include_top=False, input_shape=(256, 256, 3))
#base_model = DenseNet121(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
base_model = DenseNet169(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = DenseNet201(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = ResNet50(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))

## Data Augmentation
Alle der zuvor genanten Modelle wurden mit und ohne Dataaugmentation getestet. Ziel war durch das erhöhen der Varianz in den Klassen das Ergebnis zu erhöhen.
Es wurden folgende Augmentations getestet:
1. Zufällige Rotation des Bildes
2. Zufälliges Heran-Zoomen
3. Zufälliges horizontales Spiegeln
4. Zufällige Kontraständerungen
5. Zufällige Helligkeistveränderung

Im Falle des finalen Netzes haben ausnahmslos alle diese Maasnahmen zu Verschlechterungen des Ergebnises geführt und wurden daher verworfen.

In [ ]:
# Daten Augmentation
data_augmentation = tf.keras.Sequential([
    #tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    tf.keras.layers.RandomFlip(mode = 'horizontal'),
    tf.keras.layers.RandomContrast(0.4)
])

## Modelkonstruktion
Bei der Konstruktion des Gesamtmodells wurden mehrere Dinge getestet. Zunächst ein wie bereits erwähnten und verworfenen Data-Argumentation-Layer, sowie mehrere Denselayer auf der Ausgangsseite des Modells und ein Denselayer mit der Anzahl der Klassen als Units da das Original DenseNet 1000 Klassen erwartet. Die zusätzlichen Denselayer welche nicht der Klassenanzahl entsprechen wurden aufgrund von Verschlechterung des Ergebnisses wieder verworfen.
Außerdem konnten GlobalAveragePooling2D und BatchNormalization keine Verbesserung erziehlen. Hier konnte man erkennen, dass beim Testen von verschiedenen Netzen verschiedene Parameter zu verbesserungen bzw. Verschlecheterungen beitragen können. Bspw. konnte die Dataaugmentation beim ResNet ein besseres Ergebniss erzielen, wobei sie beim DenseNet das Ergebnis verschlechtert. **Anmerkung: Die pro Netz test können auf alle Parameter bezogen werden daher gehört das meiner Meinung nach entweder zur Netzauswahl oder in den Schluss zur Diskussion**

In [ ]:
# Data Augmentation erweitern
model = Sequential([
    #data_augmentation,
    base_model,
    #GlobalAveragePooling2D(),
    #BatchNormalization(),
    #Dense(256, activation="relu"),
    #Dense(128, activation='relu'),
    #Dense(64, activation="relu"),
    Dense(num_classes, activation='softmax')
])
base_model.layers[0].trainable = False

## Anpassbare Lernrate
Die Lernrate ist ein wichtiger Parameter. Im allgemeinen je geringer die Lernrate umso besser das Lernen**Anmerkung: Vieleicht etwas zu allgemein und nicht sicher ob das so überhaupt richtig ist, aber Diskutabel**. Ist die Lernrate von Anfang an sehr gering dauert das Lernen sehr lange. Meist ist aber zu Begin eine geringe Lernrate nicht nötig. Um die Lernrate je Epoche anzupassen wird die Callbackfunktion **ReduceLROnPlateau** verwendet. Diese reduziert die Lernrate wenn die valdation Accuracy sich 3 mal hintereinader nicht verbessert.
So kann zu Begin mit einer vergleichsweise hohen Lernrate gestartet werden. Die Startlernrate wurde ebenfalls empirisch bestimmt. Die Startlernrate von 0,001 hat hier keine guten Ergebnisse erziehlt.

Das Resultat dieser Vorgehensweise ist sehr positiv und hat das Ergebniss enorm verbessert.

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, min_lr=1e-18)
optimizer = Adam(learning_rate=1e-4)

## Vorzeitiger Stopp
Um ein zu langes Lernen und Overfitting zu verhindern, kann das Trainieren durch einen vorzeitigen Stopp abgebrochen werden. 

Erziehlt das Model in mehreren Epochen keine Verbesserung ist die Gefahr in ein Overfitting zu gelangen sehr groß und ein Abrechen des Lernens ist sinnvoll.

Die zu beobachtende Metrik um die Verbesserung abzuschätzen ist die val_accuracy. Diese gilt es zu maximieren somit ist der Modus **max** zu wählen. Durch den Parameter **patience** wird nach 10 Epochen ohne Verbesserung das Lernen abgebrochen. Dieser Parameter wurde nich optimiert da durch **restore_best_weights** die Gewichte der nach der gewählten Metrik das beste Ergebniss liefern.

In [ ]:
EarlyStopping_Cb = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 10, restore_best_weights = True, verbose = 1)

## Modellkompilierung

Die Modellkompilierung kann durch die Wahl der zu optimierenden Metriken verbessert werden. 
In diesem Fall führt das Optimieren des Accuracy und dem Mean-Scuared Error zur verbesserung des Ergebnisses. 

## Modelltraining

Die Anzahl der vorgegebenen Epochen wird hier 50 gesetzt da das Lernen innerhalb dieser durch die Callbackfunktion frühzeiting gestoppt wird.

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', 'mse'])

# Trainingsparameter anpassen
batch_size = 32
epochs = 50
# Modelltraining mit erweitertem Data Augmentation
model.fit(train_ds,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_ds,
          callbacks=[reduce_lr, EarlyStopping_Cb])

## Prediction und Speichern der Ergebnisse
Für die Prediction wird die Liste der Validierungsbilder iterativ in des Modell gegeben. Dabei wird erst der Inhalt des Verzeichnisses in eine Liste übertragen. Danach wird über diese Liste iteriert und dem Modell zur Vorherrsage übergeben. Die vorhergesagten Labels werden in der gleichen Reihenfolge wie die Bilder eingelsen wurden in einem Numpy-Array gespeichert. Die Liste der Bildnamen und der vorhergesagten Labels werden in einem Datenframe von Pandas kombiniert und anschießend in einer CSV-Datei exportiert.

In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = os.listdir('./test_data')
class_names = sorted(os.listdir(train_directory))
label_predictions = []
img_array = []
test_directory = 'test_data'


# Durchlaufe das Testverzeichnis und lade die Bilder
for image_name in image_names:
    url = "./test_data/" + image_name
    img = tf.keras.utils.load_img(url, target_size=image_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label_predictions.append(class_names[np.argmax(score)])


# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

## Ergebnisanalyse

### Was kannnoch besser gemacht werden?

Kombinieren des Models mit einer Texterkennung, außerdem könnte mehr Aufwand in der Bildvorverarbeitung ebenfalls Auswirkungen auf ein besseres und vieleicht auch ein robusteres Ergebniss erzielen testen beispielsweise den Kontrast bei allen Bildern erhöhen.


### Welche Erkenntnisse konnten durch die Arbeit gewonnen werden?
Es ergeben sich einige Punkte, welche in dieser Arbeit bemerkbar wurden. 

* Jeder Modellansatz muss für sich betrachtet werden. Meist können keine Schlüsse von anderen Modellen auf das aktuelle Modell geschlossen werden. (Wie z.B. Dataaugmentation, GlobalAveragePooling2D und BatchNormalization diese haben bei den Netzen ResNet und EffincentNet zu Verbesserungen geführt)
* Durch die Menge an Daten wurde man gezwungen mit datasets zu arbeiten **Anmerkung: Hört sich etwas rupig an das man gezwungen wird, Vielleicht eher die beschreiben als Notwendigkeit für effizienteres Laden der Daten aufgrund begranzten GRakaSpeichers**, welche viele Möglichkeiten bieten Verbesserungen sowie Vereinfachungen zu erreichen
* Es müssen nicht immer eigene, aufwändige Ansätze zielführend sein. Meist sind gegebene Ansätze, wie die in der Keras-Bibliothek ausreichend für das gegebene Problem oder wie in diesem Fall sogar besser.

